In [2]:
import os
import pandas as pd

#### 유저 데이터

In [3]:
# 사용자 u.user파일을 DataFrame으로 읽기
base_src = "./"
u_user_src = os.path.join(base_src, "u.user")
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src, sep='|', 
                    names=u_cols, 
                    encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


#### 아이템 데이터

In [4]:
# u.item 파일을 DataFrame으로 읽기
u_item_src = os.path.join(base_src,'u.item')
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL','unknown','Action','Adventure','Animation',
          'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src,
	    sep='|',
            names=i_cols,
            encoding='latin-1')
movies.head()


,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


#### 평가 데이터

In [5]:
# u.data 파일을 DataFrame으로 읽기
u_data_src = os.path.join(base_src,'u.data')
r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
        sep = '\t',
        names = r_cols,
        encoding='latin-1')

#### 타임 스탬프 제거
- 데이터가 아주 오래된 형식(1970년대 타임스탬프)이라 변환 시 에러가 나거나
- 일부 값이 깨져서 숫자가 너무 크거나 작을 경우 → Pandas가 datetime으로 변환하지 못함.
- 불필요한 타임스탬프 열을 제거

In [6]:
from IPython.display import display

# ratings DataFrame에서 timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]
display(ratings.head())
display(movies.head())

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


#### 데이터 분리 라이브러리
- 데이터셋을 trainSet, testSet으로 분리해주는 데이터 분리 라이브러리

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

# 평가 데이터 카피(원본 보존)
x = ratings.copy()          # 전체 평점 데이터를 x에 복사
y = ratings['user_id']      # y는 사용자 ID만 추출 (라벨 역할)

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.25,  # train: 75% test: 25
                                                    stratify=y)      # # y(즉, user_id)의 분포를 train/test에 동일하게 유지

# ----------------------------------------------------------
# ✅ RMSE (Root Mean Squared Error, 평균제곱근오차)
# ----------------------------------------------------------
# 예측값(y_pred)과 실제값(y_true)의 차이를 계산하여
# 평균 오차의 크기를 나타내는 함수입니다.
# ➜ 값이 작을수록 모델의 예측이 실제값에 더 가깝습니다.
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# ----------------------------------------------------------
# ✅ 모델 성능 평가 함수 (score)
# ----------------------------------------------------------
# 주어진 모델이 테스트 데이터(x_test)에서
# 실제 평점과 얼마나 비슷하게 예측하는지를 RMSE로 평가합니다.
# 
# 1. x_test에서 (user_id, movie_id) 쌍을 추출
# 2. 각 쌍에 대해 model(user, movie)로 예측 평점을 계산
# 3. 실제 평점(y_true)과 비교하여 RMSE 계산
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# best_seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()

def best_seller(user_id, movie_id):
    try:
        # movie_id가 여긴 없고 테스트 데이터에만 있을 수 있다 -> 인덱스 에러 대비
        rating = train_mean[movie_id]
    except:
        # movie_id가 없으면 기본값 3.0을 주겠다
        rating = 3.0
    return rating

score(best_seller)

np.float64(1.0261090004975557)

#### 남녀별로 평균 평점을 따로 계산해서, 성별에 따라 다른 예측값을 사용

In [11]:
# ----------------------------------------------------------
# ✅ 성별에 따른 예측값 계산 
# ----------------------------------------------------------

# x_train(평점 데이터)와 users(사용자 정보)를 user_id 기준으로 병합
# → 각 평점 데이터에 사용자 성별(sex) 정보를 추가
merged_ratings = pd.merge(x_train, users)

# users 테이블의 인덱스를 user_id로 변경
# → 나중에 users.loc[user_id] 형태로 사용자 정보(예: 성별) 조회 가능
users = users.set_index('user_id')

# 영화별(movie_id) + 성별(sex) 그룹으로 묶어서 평균 평점을 계산
# → 각 영화에 대해 남성과 여성의 평균 평점을 따로 구함
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
g_mean

# 결과 예시:
# movie_id  sex
# 10        M      3.8   ← 남성이 영화 10에 준 평균 평점
# 10        F      4.2   ← 여성이 영화 10에 준 평균 평점
# 11        M      3.5
# 11        F      4.1
# ...
# 이렇게 영화별 + 성별별 평균 평점을 저장한 표가 g_mean


movie_id  sex
1         F      3.756410
          M      3.869048
2         F      3.071429
          M      3.178947
3         F      2.363636
                   ...   
1676      M      2.000000
1677      F      3.000000
1680      M      2.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3024, dtype: float64

#### 사용자-영화 평점 데이터 행렬
- 각 유저가 각영화에 몇 잠을 주었는가

In [12]:
rating_matrix = x_train.pivot(index='user_id',
                              columns='movie_id',
                              values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1668,1669,1671,1673,1675,1676,1677,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,3.0,5.0,4.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Gender 기반 추천
def ch_gender(user_id, movie_id):
    if movie_id in rating_matrix.columns:           # ① 영화가 데이터셋에 존재하면
        gender = users.loc[user_id]['sex']          # ② 해당 사용자의 성별을 조회
        if gender in g_mean[movie_id].index:        # ③ 그 성별의 평균 평점이 존재하면
            gender_rating = g_mean[movie_id][gender]  # ④ 그 평균 평점을 사용
        else:
            gender_rating = 3.0                     # ⑤ 없으면 기본값 3.0 (중간값)
    else:
        gender_rating = 3.0                         # ⑥ 영화 데이터가 없으면 기본값 3.0
    return gender_rating                            # ⑦ 예측 평점 반환

score(ch_gender)

np.float64(1.031994875263707)